In [2]:
from os.path import exists

In [3]:
def run(data_size, aug_type, ep=3):
    if exists('./checkpoints'):
        !rm -r ./checkpoints
    
    if aug_type == 'reda':
        train_set_suffix = f"_{data_size}_aug_reda"
    else:
        train_set_suffix = f"_{data_size}_aug_reda_ngram"

    train_set_path = './train' + train_set_suffix + '.txt'

    !python -u -m paddle.distributed.launch --gpus "0" train_pointwise.py \
            --train_set_path {train_set_path} \
            --dev_set_path './dev.txt' \
            --epochs {ep} \
            --save_dir ./checkpoints\
            --batch_size 64 \
            --learning_rate 2E-5 \
    
    model = !ls ./checkpoints/
    model.sort()
    model_path = f"./checkpoints/{model[-1]}/model_state.pdparams"

    !python -u -m paddle.distributed.launch --gpus "0" \
            predict_pointwise.py \
            --out_dir './test_preds/' \
            --train_set_suffix {train_set_suffix} \
            --device gpu \
            --params_path {model_path}\
            --batch_size 64 \
            --max_seq_length 128 \
            --input_file 'test.txt'
    
    !rm train_set_path

In [4]:
run('5k', 'reda')
run('5k', 'reda_ngram')

-----------  Configuration Arguments -----------
backend: auto
elastic_server: None
force: False
gpus: 0
heter_worker_num: None
heter_workers: 
host: None
http_port: None
ips: 127.0.0.1
job_id: None
log_dir: log
np: None
nproc_per_node: None
run_mode: None
scale: 0
server_num: None
servers: 
training_script: train_pointwise.py
training_script_args: ['--train_set_path', './train_5k_aug_reda.txt', '--dev_set_path', './dev.txt', '--epochs', '3', '--save_dir', './checkpoints', '--batch_size', '64', '--learning_rate', '2E-5']
worker_num: None
workers: 
------------------------------------------------
WARNING 2021-12-05 14:30:19,426 launch.py:416] Not found distinct arguments and compiled with cuda or xpu. Default use collective mode
launch train in GPU mode!
INFO 2021-12-05 14:30:19,428 launch_utils.py:527] Local start 1 processes. First process distributed environment info (Only For Debug): 
    +=======================================================================================+
    |